# Homework #5: Classification
# 2. Predicting Party Affiliation by Votes

So far in our classification assignments, we have provided most of the code for extracting features.  Now it is your turn to get your hands dirty and implement feature extraction from data.  For this problem, we will use the `congress` database, first introduced in Homework #2.  You should still have this database on your VM (you can check it with `psql congress`); if not, follow the instructions in Homework #2 to recreate this database.

Your task for this problem is to predict, using the votes in the House cast by a Representative in the 2015 session, whether this Representative is Republican.  Under the `congress/` subdirectory, the script `classify.py` is ready to go. It uses Python `sklearn`'s multivariate Bernoulli Naive Bayes classification, which models each Representative's voting history as bit vector, where each component corresponds to what the Reprsentative did for each vote.  Your job is complete this notebook file.  What your code needs to produce is explained by the comments in the following cells.

The program will first run a particular train-test split to print out more specific information (for debugging); then it will run 10-fold cross validation on the entire dataset. If the dataset that you have prepared is too small, you may get some error like 'All the n_labels for individual classes are less than 10 folds.' just like what the example codes below will show.

In [1]:
import sys
import psycopg2
import cPickle as pickle
import numpy

/home/vcm/.local/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
pikfile = 'prepared.pik'
def get_data():
    with open(pikfile, 'rb') as f:
        return pickle.load(f)

In [3]:
def prepare_data():
    # Connect to the congress database and construct the following objects:
    # votes: a list of vote ids (from the id column of votes table) for votes
    #        of interest.
    # persons: a list of person ids for legislators (from the person_id column
    #          of the person_votes table) who ever casted any vote of interest.
    # X: a matrix (numpy.array) where each row corresponds to a person in
    #    persons above and each column corresponds to a vote in votes
    #    above; X[i,j] = 1 if the the vote cast by persons[i] for
    #    votes[j] is positive, or 0 otherwise.
    # y: a vector (numpy.array) where each component corresponds to a person
    #    in persons above; y[i] = 1 if the person is Republican, or 0 otherwise.

    conn = psycopg2.connect("dbname=congress")
    cur = conn.cursor()
    
    # correct implementation to get votes
    cur.execute('''
SELECT id
FROM votes
WHERE chamber = 'h' AND session = 2015
ORDER BY id;
''')
    
    #votes array of voteIDS

    votes = [ vote_id for vote_id, in cur ]
    vote2index = { vote_id : i for i, vote_id in enumerate(votes) }

    # REPLACE THE FOLLOWING (INCORRECT) WITH YOUR IMPLEMENTATION:
    # You will need to get persons and their party from the database to replace the hard coded samples
    

    #persons array of personIDs
    
    #SQL query for list of person_ids | party 
        
    cur.execute('''
SELECT DISTINCT p.person_id, r.party
FROM person_votes p, person_roles r, votes v
WHERE v.id = p.vote_id AND r.person_id = p.person_id AND v.chamber = 'h' AND v.session = 2015
AND r.start_date <= '2015-12-31' AND r.end_date >= '2015-12-31'
ORDER BY p.person_id;
''')
   
    persons = [] 
    y = []
    
    for person_id, party in cur:
        persons.append(person_id)
        
        if party == 'Republican': 
            y.append(1)
        else: 
            y.append(0)
        
    person2index = { p_id : i for i, p_id in enumerate(persons) }
    
    # REPLACE THE FOLLOWING (INCORRECT) WITH YOUR IMPLEMENTATION:
    # You will need to set up the features

# X 

    X = numpy.zeros((len(persons), len(votes)), dtype=int)

    cur.execute('''
SELECT p.person_id, p.vote_id, p.vote  
FROM person_votes p, votes v, person_roles r
WHERE v.chamber = 'h' AND v.session = 2015 AND v.id = p.vote_id AND r.person_id = p.person_id
AND r.start_date <= '2015-12-31' AND r.end_date >= '2015-12-31'
ORDER BY v.id, p.person_id
''')
    
    for person_id, vote_id, vote in cur:            
        if (vote == 'Yea' or vote == 'Aye'): #positive vote: Yea aye
            X[person2index[person_id], vote2index[vote_id]] = 1
        else: #negative vote
            X[person2index[person_id], vote2index[vote_id]] = 0
                
    # REPLACE THE FOLLOWING (INCORRECT) WITH YOUR IMPLEMENTATION:
    # Instead of hard coding, fill in X matrix with general information about person index and vote index
    # An example that person A000055 votes positively for the first vote_id
   
    
    #person_id = 'A000055'
    #vote_id = votes[0]
    #X[person2index[person_id], vote2index[vote_id]] = 1
    
    # close database connection
    cur.close()
    conn.close()

    # DO NOT MODIFY THE REST OF THE CODE:
    with open(pikfile, 'wb') as f:
        pickle.dump((X, y, persons, votes), f)


In [4]:
prepare_data()
%run -i classify.py

/home/vcm/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


434 instances, 704 features
debugging on one train/test split:
train accuracy: 1.0000
test accuracy: 1.0000
top 20 features:
	h491-114.2015       : -4.8853	h558-114.2015       : +5.0601
	h492-114.2015       : -4.8853	h422-114.2015       : +5.0601
	h554-114.2015       : -4.8853	h421-114.2015       : +5.0601
	h566-114.2015       : -4.8853	h357-114.2015       : +5.0601
	h180-114.2015       : -4.8789	h414-114.2015       : +5.0525
	h181-114.2015       : -4.8789	h354-114.2015       : +5.0525
	h379-114.2015       : -4.8789	h564-114.2015       : +5.0449
	h392-114.2015       : -4.8789	h523-114.2015       : +5.0449
	h541-114.2015       : -4.8789	h457-114.2015       : +5.0449
	h542-114.2015       : -4.8789	h446-114.2015       : +5.0449
running 10-fold cross validation...
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
accuracy: 1.0000 (+/- 0.0000)
